# Practicum observer pattern

## Doel
- Kennismaken met het observer pattern

## Crypto currencies
Deze opgave gaat over de koersen van crypto currencies.

De waarde van cryptomunten (zoals bitcoin) verandert voortdurend. In realtime koersoverzichten is te zien dat de koers van cryptomunten voortdurend verandert (fluctueert).

In deze opgave staan twee classes centraal: `Coin` en `Market`.

De klasse `Coin` representeert een cryptomunt. De klasse `Market` bevat cryptomunten en zorgt ervoor fluctuatie wordt gesimuleerd.

Verder is er een klasse `Main`, die er voor zorgt dat alle klassen worden gemaakt en met elkaar worden gekoppeld.

````{note} Simulatie
De simulatie van de cryptomarkt in deze opgave is een sterk vereenvoudige versie van de werkelijkheid. Zo is in werkelijkheid de ene cryptomunt meer volatiel (veranderlijkheid van koers) dan de andere, terwijl in deze simulatie de cryptomunten op dezelfde wijze (willekeurig) van koers veranderen.
````


## De klasse `Coin`

Maak een nieuw project. Maak binnen het project een package met de naam **crypto**.

Maak in de package crypto de klasse `Coin` met de volgende inhoud:
```Java
package crypto;

import java.util.Random;

public class Coin {

    private String name;
    private double price;
    private static Random random = new Random();

    public Coin(String name, double price) {
        this.name = name;
        this.price = price;
    }

    public String getName() {
        return name;
    }

    public double getPrice() {
        return price;
    }
    
    public void fluctuate(double norm) {
        norm+=0.05-random.nextDouble()*0.10;
        price+=price*norm;
    }
    
}
```

De klasse `Coin` heeft twee instantie-variabelen: **name** en **price**. De variabele **name** is de naam van de munt en **prijs** de prijs (waarde) in euro's.

Ze worden geïnitialiseerd via de constructor en kunnen worden opgevraagd via getters.

De methode **fluctuate** verandert willekeurig de koers. De parameter **norm** bevat de normale koersverandering, een fractie. Daar wordt willekeurig een aanpassing van -0.05 tot 0.05 aan gedaan.

## De klasse `Market`

De klasse `Market` is een gesimuleerde cryptomarkt.

Voeg deze klasse toe aan het project, aan de hand van onderstaande code:
```Java
package crypto;

import java.util.ArrayList;
import java.util.Collections;
import java.util.List;
import java.util.Random;

public class Market implements Runnable {

    private boolean stop;
    private List<Coin> coins;
    private Random random = new Random();

    public Market() {
        stop=false;
        coins=Collections.synchronizedList(new ArrayList<>());
        new Thread(this).start();
    }

    /*
    Coins toevoegen en verwijderen
     */

    public void addCoin(Coin coin) {
        coins.add(coin);
    }

    public void removeCoin(Coin coin) {
        coins.remove(coin);
    }

    /*
     Gegevens van aanwezige coins opvragen
     */
    public int getNumberOfCoins() {
        return coins.size();
    }

    public String getCoinName(int i) {
        return coins.get(i).getName();
    }

    public double getCoinPrice(int i) {
        return coins.get(i).getPrice();
    }

    /*
    Methodes om de prijs van de coins te laten fluctueren
     */

    private void fluctuate() {
        double norm=0.05-random.nextDouble()*0.10;
        for(Coin coin : coins) {
            coin.fluctuate(norm);
        }
    }

    public void stop() {
        stop=true;
    }

    @Override
    public void run() {
        while(!stop) {
            fluctuate();
            try {
                Thread.sleep(300+random.nextInt(1700));
            } catch (InterruptedException e) {
                e.printStackTrace();
                stop();
            }
        }
    }

}
```

De belangrijkste instantie-variabele is `coins`. Deze bevat een lijst van aanwezige coins in de market. Met de methoden `addCoin` en `removeCoin` kunnen deze worden toegevoegd en verwijderd.

Informatie over de coins op de market kan opgehaald worden met de methodes `getNumberOfCoins`, `getCoinName` en `getCoinPrice`. 

De methode `fluctuate` laat de koersen fluctueren door de methode `fluctuate` van elke coin aan te roepen.

Meest complexe deel van de klasse `Market` is de constructor.

De regel 
```java
new Thread(this).start();
```
zorgt ervoor dat een *thread* wordt aangemaakt met het `Market`-oject (*this*). Deze thread wordt direct gestart.

De regel
```java
coins=Collections.synchronizedList(new ArrayList<>());
```
maakt een *thread safe* `ArrayList` aan. Deze heeft exact dezelfde functionaliteit als een reguliere arraylist, maar is geschikt om binnen een multi threaded programma te gebruiken.

De `run`-methode bevat de code die wordt uitgevoerd door de *thread*. Deze methode bevat een oneindige loop. Als eerste wordt de methode `fluctuate` aangeroepen. Vervolgens wordt een pauze ingelast met een willekeurige tijdsduur tussen 0,3 en 2,0 seconden. Daardoor zijn niet alleen de omvang van de koersveranderingen willekeurig, maar ook het moment waarop de koers verandert.


## De klasse `Main`

Maak een klasse `Main` met een *main-methode*.
    
Met de onderstaande code wordt de market gemaakt, drie cryptomunten en worden deze munten toegevoegd aan de market:
```java
    Market market = new Market();
    market.addCoin(new Coin("BTC Bitcoin", 51000));
    market.addCoin(new Coin("LTC Litecoin", 183));
    market.addCoin(new Coin("XRP Ripple", 0.8684));
```

Als dit programma wordt uitgevoerd, blijft het programma doorlopen, ook nadat de main-methode is beëindigd. Dit komt doordat op de achtergrond de *thread* draait van de market. Het programma kan alleen handmatig gestopt worden (het gaat immers om een oneindige loop).

De main-methode kan worden uitgebreid met code dat periodiek de munten en koersen weergeeft:
```java
    while (true) {
        // Toon de coins en prijzen
        System.out.println("=========");
        for(int i=0; i<market.getNumberOfCoins(); i++) {
            System.out.println(market.getCoinName(i) + ": € " + market.getCoinPrice(i));
        }
        // Een halve seconde wachten
        try {
            Thread.sleep(500);
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
    }
```

Kijk goed naar de koersen. Wat valt op? Worden de koersen correct weergegeven?

## Observer pattern

Het programma bevat twee threads: De market, die voor de koersschommelingen zorgt, en de main-thread, die de koersen weergeeft. 

Er is geen communicatie tussen de threads. Daarom worden koersveranderingen niet onmiddellijk weergegeven. Of worden de koersen opnieuw weergegeven terwijl er geen sprake is van koersverandering.

Een mogelijke oplossing voor dit probleem is de code om koersen weer te geven toevoegen aan `Market`, of om de methode om de koersen weer te geven vanuit `Market` aanroepen.

Dit is echter niet gewenst. De `Market` zou volledig onafhankelijk moeten kunnen functioneren van weergave.

Het observer pattern biedt de mogelijkheid voor *loose coupling* tussen `Market` en weergave.

Daarbij is de klasse om weergave te regelen een *observer*, en is de `Market` *observable* (ook wel *subject* genoemd).

In vier stappen wordt het bestaande project aangepast aan het observer pattern.

## Stap 1: Interfaces maken

Maak de interfaces voor de observer en observable (subject).

De interface `Subject` bevat de volgende methode-definities:
```java
    public void registerObserver(Observer o);
    public void removeObserver(Observer o);
    public void notifyObservers();
```

De interface `Observer` bevat de volgende methode-definitie:
```Java
    public void update(Market market);
```

## Stap 2: Van klasse `Market` een observable maken

- Laat de klasse `Market` de interface `Subject` implementeren
- Voeg de volgende klasse-variabele toe aan `Market`:
```Java
private List<Observer> observers = new ArrayList<>();
```
- Schrijf implementaties voor de methoden `registerObserver` en `removeObserver`. Dit zijn eenvoudige methodes, om een observer toe te voegen aan of te verwijderen uit de lijst `observers`.
- Schrijf de implementatie voor de methode `notifyObservers`. Deze methode bestaat uit een loop door de lijst `observers`. In de loop wordt de methode `update` van elke observer aangeroepen, met argument *this*.

Test of het programma nog functioneert. Als het goed is, heeft de toegevoegde functionaliteit geen invloer op de werking van het programma.


## Stap 3: Een observer maken

Maak een klasse met de naam `CryptoConsole`, die de interface `Observer` implementeert.

Schrijf de implementatie voor de methode `update`.

Deze methode bevat de code om de munten en koersen weer te geven die al in de main-methode staat:
```java
    System.out.println("=========");
    for(int i=0; i<market.getNumberOfCoins(); i++) {
        System.out.println(market.getCoinName(i) + ": € " + market.getCoinPrice(i));
    }
```


## Stap 4: De main-methode aanpassen

De loop in de main-methode is overbodig geworden. Verwijder deze volledig. Alleen de code om de market en de coins te maken, blijft staan.

De loop wordt vervangen door code waarmee de observer wordt gemaakt en gekoppeld aan de market:
```java
CryptoConsole cryptoConsole = new CryptoConsole();
market.registerObserver(cryptoConsole);
```

Test het programma. Hoe is de weergave? Wat is het verschil met de voorgaande situatie?



## Een extra observer

Naast *loose coupling* tussen weergave van de coins en de market biedt het observer pattern ook de mogelijkheid om verschillende, onafhankelijk van elkaar functionerende, weergaven te maken.

De klasse `CryptoAlltimeHighNotification` is een observer die van een bepaalde coin een melding op de console afdrukt als de koers een recordhoogte bereikt.

Bijvoorbeeld in het geval van Bitcoin:
```
Alltime high!! BTC Bitcoin € 65238.094771831886
```

Een instantie van `CryptoAlltimeHighNotification` moet één specifieke coin in de gaten houden.

In de huidige versie van de klasse `Market` worden coins benaderd aan de hand van een index. Dit is echter geen *unieke identifier* voor een coin. De index kan veranderen als een coin verwijderd wordt.

Elke cryptomunt heeft een eigen afkorting. Deze wordt ook wel *code* genoemd. Zo is de afkorting BTC voor Bitcoin, en XRP voor Ripple.

Dit is een geschikte *unieke identifier*.

Om munten te vinden op basis van afkorting, moeten eerst de klassen `Coin` en `Market` worden aangepast.

## Code als unieke identifier

Voeg de klasse variabele `code`, type `String` toe aan de klasse `Coin`.

Pas de constructor aan, zodat deze de volgende definitie krijgt:
```Java
public Coin(String code, String name, double price)
```

Maak een *getter* voor `code`.

Voeg aan de klasse Market de volgende methodes toe:
```Java
    public String getCoinName(String code)
    public double getCoinPrice(String code)
```

De volgende hulp-methode kan handig zijn:
```Java
    private Coin findCoinByCode(String code)
```

Tot slot dienen er nog kleine aanpassingen te worden gedaan aan de code die de instanties maakt van `Coin`.

## De klasse `CryptoAlltimeHighNotification`

Maak de klasse `CryptoAlltimeHighNotification`. Deze implementeert de interface `Observer`.

De klasse `CryptoAlltimeHighNotification` heeft de volgende klasse-variabelen:
```java
    private String code; // afkorting van de munt
    private double highestPrice;
```

De variabele `code` wordt door middel van de constructor geïnitialiseerd.

Schrijf de implementatie van de methode `update`.
Deze methode vraagt de prijs op van de coin, en controleert of deze hoger is dan `highestprice`. Zo ja, print op de console een melding en actualiseer de waarde van `highestprice`.

Bijvoorbeeld in het geval van Bitcoin:
```
Alltime high!! BTC Bitcoin € 65238.094771831886
```

Om deze observer te laten functioneren, dient in de main-methode een instantie gemaakt te worden.

Maak een instantie met code BTC (bitcoin) en koppel deze aan market.


## Standaard klassen voor het observer pattern gebruiken
Java bevat standaard ondersteuning voor het observer-pattern door middel van de interface `Observer` en de klasse `Observable`.

Documentatie:
https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/util/Observer.html
https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/util/Observable.html

````{note}
Hoewel `java.util.Observer` en `java.util.Observable` vanaf Java 9 worden beschouwd als *deprecated*, is het een goede oefening om het observer pattern ook met deze generieke interface en klasse te proberen. De reden dat ze *deprecated* zijn is dat men van mening is dat het originele observer pattern te beperkt is. Zo biedt het geen standaard regels als het gaat om de volgorde waarin de update-methode van observers wordt aangeroepen.
````

Maak een kopie van de package crypto. Verwijder in de kopie de interfaces Observer en Subject.

Laat de klasse `Market` de klasse **Observable** (uit *package* `java.util`) extenden.

Verwijder de code voor het observer-pattern (de variabele `observers` en de methodes `registerObserver`, `removeObserver` en `notifyObservers`) uit `Market`.

Laat de klasse `CryptoConsole` de interface `Observable` (uit *package* `java.util`) implementeren.

De methode `update` uit `java.util.Observer` heeft een andere definitie. Deze is meer generiek. Implementeer de juiste methode update.

Het market-object kan uit de parameter `observable` gehaald worden met behulp van een cast:
```java
Market market = (Market)observable;
```

Ook `CryptoAlltimeHighNotification` dient aangepast te worden.

Tenslotte nog twee kleine aanpassingen om het werkend te krijgen:
- In plaats van `registerObserver` bevat `java.util.Observable` de methode `addObserver`.
- Om er voor te zorgen dat de update-methode van de observers wordt aangeroepen, moet vóór het aanroepen van `notifyObservers` in `Market` de methode `setChanged` aangeroepen worden!

Als aanroep van `setChanged` wordt vergeten, dan lijkt het alsof het hele observer pattern om onverklaarbare redenen niet functioneert!